In [0]:
#filename = dbutils.widgets.get('filename')
filename='orders .csv'
print("Selected file name:", filename)


Selected file name: orders .csv


In [0]:
databricksScope='secretscope'
secret_value=dbutils.secrets.get(scope=databricksScope,key='databricskaccesstoken')
print(secret_value)

[REDACTED]


In [0]:
already_mounted=False
for i in dbutils.fs.mounts():
    if i.mountPoint=='/mnt/sales':
        already_mounted=True
        print('alreadymounted')
        break
    else:
        already_mounted=False
print(already_mounted)


alreadymounted
True


In [0]:
if not already_mounted:
  dbutils.fs.mount(
  source='wasbs://customeranalyticscontainer@customeranalyticssa.blob.core.windows.net',
  mount_point='/mnt/sales',
  extra_configs={'fs.azure.account.key.customeranalyticssa.blob.core.windows.net': secret_value })
  print('just mounted')
else:
  print('already mounted')


already mounted


In [0]:
# Example: Accessing a secret within a notebook
#key = dbutils.secrets.get(scope="secretscope", key="databricskaccesstoken")
dbutils.secrets.get(scope=databricksScope,key='databricskaccesstoken')
print(secret_value)

[REDACTED]


In [0]:
orders_df=spark.read\
.format('csv')\
.option('header','true')\
.option('inferSchema','true')\
.load('dbfs:/mnt/sales/landing/{}'.format(filename))

In [0]:
orders_df.createOrReplaceTempView('orders')

In [0]:
display(orders_df.select('order_status').distinct())

order_status
PENDING_PAYMENT
COMPLETE
ON_HOLD
PAYMENT_REVIEW
PROCESSING
CLOSED
SUSPECTED_FRAUD
PENDING
CANCELED


In [0]:
orders_df_count=orders_df.count()
display(orders_df_count)

68883

In [0]:
flag=False
orders_df_count=orders_df.count()
distinct_order_id=orders_df.select('order_id').distinct().count()
if orders_df_count!=distinct_order_id:
    flag=True
if flag:
    dbutils.fs.mv('dbfs:/mnt/sales/landing/{}'.format(filename),'dbfs:/mnt/sales/bin/')
    dbutils.notebook.exit({'error':'duplicate_records','exit':'staledata'})


In [0]:
dbServer='swaroops020'
dbPort='1433'
dbName='swaroops020'
dbUser='swaroops020'
dbPassword='sqlpassword'
databricksScope='secretscope'
connectionUrl='jdbc:sqlserver://{}.database.windows.net:{};database={};user={};'.format(dbServer,dbPort,dbName,dbUser)
dbPassword=dbutils.secrets.get(scope=databricksScope,key='sqlpassword')
connectionProperties={
'password':dbPassword,'driver':'com.microsoft.sqlserver.jdbc.SQLServerDriver'}

In [0]:
validStatusDf=spark.read.jdbc(url=connectionUrl,table='dbo.order_status',properties=connectionProperties)

In [0]:
validStatusDf.createOrReplaceTempView('order_status')

In [0]:
orders_df_status=spark.sql('select count( *) from orders where order_status in (select * from order_status)')
display(orders_df_status)

org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 222.0 failed 4 times, most recent failure: Lost task 0.3 in stage 222.0 (TID 243) (10.139.64.4 executor driver): com.databricks.sql.io.FileReadException: Error while reading file dbfs:/mnt/sales/landing/orders%20.csv. [DEFAULT_FILE_NOT_FOUND] It is possible the underlying files have been updated. You can explicitly invalidate the cache in Spark by running 'REFRESH TABLE tableName' command in SQL or by recreating the Dataset/DataFrame involved. If disk cache is stale or the underlying files have been removed, you can invalidate disk cache manually by restarting the cluster. SQLSTATE: 42K03
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1$$anon$2.logFileNameAndThrow(FileScanRDD.scala:703)
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1$$anon$2.getNext(FileScanRDD.scala:641)
	at org.apache.spark.util.NextIterator.hasNext(NextIterator.scala:73)
	at org.apache.spark.sql.ex

In [0]:

if orders_df_count!=orders_df_status:
     dbutils.fs.mv('dbfs:/mnt/sales/landing/{}'.format(filename),'dbfs:/mnt/sales/bin/')
     dbutils.notebook.exit({'error':'order_status is out of range'})
else:
    dbutils.fs.mv('dbfs:/mnt/sales/landing/{}.format(filename)','dbfs:/mnt/sales/staging/')


---------------------------------------------------------------------------
ExecutionError                            Traceback (most recent call last)
File <command-4210319006168137>, line 2
      1 if orders_df_count!=orders_df_status:
----> 2      dbutils.fs.mv('dbfs:/mnt/sales/landing/{}'.format(filename),'dbfs:/mnt/sales/bin/')
      3      dbutils.notebook.exit({'error':'order_status is out of range'})
      4 else:

File /databricks/python_shell/dbruntime/dbutils.py:378, in DBUtils.FSHandler.prettify_exception_message.<locals>.f_with_exception_handling(*args, **kwargs)
    376 exc.__context__ = None
    377 exc.__cause__ = None
--> 378 raise exc

ExecutionError: An error occurred while calling o405.mv.
: java.io.FileNotFoundException: dbfs:/mnt/sales/landing/orders .csv
	at com.databricks.backend.daemon.data.client.DatabricksFileSystemV2.$anonfun$getFileStatus$2(DatabricksFileSystemV2.scala:1024)
	at com.databricks.s3a.S3AExceptionUtils$.convertAWSExceptionToJavaIOException(Data

In [0]:
order_items=spark.read\
    .format('csv')\
    .option('inferSchema','true')\
    .option('header','true')\
    .load('dbfs:/mnt/sales/order_items/order_items.csv')




---------------------------------------------------------------------------
AnalysisException                         Traceback (most recent call last)
File <command-4477688624235572>, line 5
      1 orders_df=spark.read\
      2 .format('csv')\
      3 .option('header','true')\
      4 .option('inferSchema','true')\
----> 5 .load('dbfs:/mnt/sales/landing/{}'.format(filename))

File /databricks/spark/python/pyspark/instrumentation_utils.py:47, in _wrap_function.<locals>.wrapper(*args, **kwargs)
     45 start = time.perf_counter()
     46 try:
---> 47     res = func(*args, **kwargs)
     48     logger.log_success(
     49         module_name, class_name, function_name, time.perf_counter() - start, signature
     50     )
     51     return res

File /databricks/spark/python/pyspark/sql/readwriter.py:312, in DataFrameReader.load(self, path, format, schema, **options)
    310 self.options(**options)
    311 if isinstance(path, str):
--> 312     return self._df(self._jreader.load(path))
  

In [0]:
order_items.printSchema()

---------------------------------------------------------------------------
AnalysisException                         Traceback (most recent call last)
File <command-4477688624235572>, line 5
      1 orders_df=spark.read\
      2 .format('csv')\
      3 .option('header','true')\
      4 .option('inferSchema','true')\
----> 5 .load('dbfs:/mnt/sales/landing/{}'.format(filename))

File /databricks/spark/python/pyspark/instrumentation_utils.py:47, in _wrap_function.<locals>.wrapper(*args, **kwargs)
     45 start = time.perf_counter()
     46 try:
---> 47     res = func(*args, **kwargs)
     48     logger.log_success(
     49         module_name, class_name, function_name, time.perf_counter() - start, signature
     50     )
     51     return res

File /databricks/spark/python/pyspark/sql/readwriter.py:312, in DataFrameReader.load(self, path, format, schema, **options)
    310 self.options(**options)
    311 if isinstance(path, str):
--> 312     return self._df(self._jreader.load(path))
  

In [0]:
order_items.createOrReplaceTempView('order_items_table')

---------------------------------------------------------------------------
AnalysisException                         Traceback (most recent call last)
File <command-4477688624235572>, line 5
      1 orders_df=spark.read\
      2 .format('csv')\
      3 .option('header','true')\
      4 .option('inferSchema','true')\
----> 5 .load('dbfs:/mnt/sales/landing/{}'.format(filename))

File /databricks/spark/python/pyspark/instrumentation_utils.py:47, in _wrap_function.<locals>.wrapper(*args, **kwargs)
     45 start = time.perf_counter()
     46 try:
---> 47     res = func(*args, **kwargs)
     48     logger.log_success(
     49         module_name, class_name, function_name, time.perf_counter() - start, signature
     50     )
     51     return res

File /databricks/spark/python/pyspark/sql/readwriter.py:312, in DataFrameReader.load(self, path, format, schema, **options)
    310 self.options(**options)
    311 if isinstance(path, str):
--> 312     return self._df(self._jreader.load(path))
  

In [0]:
%sql
select * from order_items_table;

---------------------------------------------------------------------------
AnalysisException                         Traceback (most recent call last)
File <command-4477688624235572>, line 5
      1 orders_df=spark.read\
      2 .format('csv')\
      3 .option('header','true')\
      4 .option('inferSchema','true')\
----> 5 .load('dbfs:/mnt/sales/landing/{}'.format(filename))

File /databricks/spark/python/pyspark/instrumentation_utils.py:47, in _wrap_function.<locals>.wrapper(*args, **kwargs)
     45 start = time.perf_counter()
     46 try:
---> 47     res = func(*args, **kwargs)
     48     logger.log_success(
     49         module_name, class_name, function_name, time.perf_counter() - start, signature
     50     )
     51     return res

File /databricks/spark/python/pyspark/sql/readwriter.py:312, in DataFrameReader.load(self, path, format, schema, **options)
    310 self.options(**options)
    311 if isinstance(path, str):
--> 312     return self._df(self._jreader.load(path))
  

In [0]:
tableName = "customers"

# Load table into DataFrame
customers_df = spark.read.jdbc(url=connectionUrl, table=tableName, properties=connectionProperties)


---------------------------------------------------------------------------
AnalysisException                         Traceback (most recent call last)
File <command-4477688624235572>, line 5
      1 orders_df=spark.read\
      2 .format('csv')\
      3 .option('header','true')\
      4 .option('inferSchema','true')\
----> 5 .load('dbfs:/mnt/sales/landing/{}'.format(filename))

File /databricks/spark/python/pyspark/instrumentation_utils.py:47, in _wrap_function.<locals>.wrapper(*args, **kwargs)
     45 start = time.perf_counter()
     46 try:
---> 47     res = func(*args, **kwargs)
     48     logger.log_success(
     49         module_name, class_name, function_name, time.perf_counter() - start, signature
     50     )
     51     return res

File /databricks/spark/python/pyspark/sql/readwriter.py:312, in DataFrameReader.load(self, path, format, schema, **options)
    310 self.options(**options)
    311 if isinstance(path, str):
--> 312     return self._df(self._jreader.load(path))
  

In [0]:
customers_df.createOrReplaceTempView('customers')

---------------------------------------------------------------------------
AnalysisException                         Traceback (most recent call last)
File <command-4477688624235572>, line 5
      1 orders_df=spark.read\
      2 .format('csv')\
      3 .option('header','true')\
      4 .option('inferSchema','true')\
----> 5 .load('dbfs:/mnt/sales/landing/{}'.format(filename))

File /databricks/spark/python/pyspark/instrumentation_utils.py:47, in _wrap_function.<locals>.wrapper(*args, **kwargs)
     45 start = time.perf_counter()
     46 try:
---> 47     res = func(*args, **kwargs)
     48     logger.log_success(
     49         module_name, class_name, function_name, time.perf_counter() - start, signature
     50     )
     51     return res

File /databricks/spark/python/pyspark/sql/readwriter.py:312, in DataFrameReader.load(self, path, format, schema, **options)
    310 self.options(**options)
    311 if isinstance(path, str):
--> 312     return self._df(self._jreader.load(path))
  

In [0]:

results_df=spark.sql("""select  orders.customer_id,customers.customer_fname,customers.customer_lname,customers.city,customers.state,count(distinct order_id) as no_of_orders,round(sum(order_item_subtotal),2) as total_value from orders join order_items_table on orders.order_id=order_items_table.order_item_order_id
join customers on orders.customer_id=customers.customer_id
group by orders.customer_id,customers.customer_fname,customers.customer_lname,customers.city,customers.state order by total_value desc""")

---------------------------------------------------------------------------
AnalysisException                         Traceback (most recent call last)
File <command-4477688624235572>, line 5
      1 orders_df=spark.read\
      2 .format('csv')\
      3 .option('header','true')\
      4 .option('inferSchema','true')\
----> 5 .load('dbfs:/mnt/sales/landing/{}'.format(filename))

File /databricks/spark/python/pyspark/instrumentation_utils.py:47, in _wrap_function.<locals>.wrapper(*args, **kwargs)
     45 start = time.perf_counter()
     46 try:
---> 47     res = func(*args, **kwargs)
     48     logger.log_success(
     49         module_name, class_name, function_name, time.perf_counter() - start, signature
     50     )
     51     return res

File /databricks/spark/python/pyspark/sql/readwriter.py:312, in DataFrameReader.load(self, path, format, schema, **options)
    310 self.options(**options)
    311 if isinstance(path, str):
--> 312     return self._df(self._jreader.load(path))
  

In [0]:
results_df.printSchema()

---------------------------------------------------------------------------
AnalysisException                         Traceback (most recent call last)
File <command-4477688624235572>, line 5
      1 orders_df=spark.read\
      2 .format('csv')\
      3 .option('header','true')\
      4 .option('inferSchema','true')\
----> 5 .load('dbfs:/mnt/sales/landing/{}'.format(filename))

File /databricks/spark/python/pyspark/instrumentation_utils.py:47, in _wrap_function.<locals>.wrapper(*args, **kwargs)
     45 start = time.perf_counter()
     46 try:
---> 47     res = func(*args, **kwargs)
     48     logger.log_success(
     49         module_name, class_name, function_name, time.perf_counter() - start, signature
     50     )
     51     return res

File /databricks/spark/python/pyspark/sql/readwriter.py:312, in DataFrameReader.load(self, path, format, schema, **options)
    310 self.options(**options)
    311 if isinstance(path, str):
--> 312     return self._df(self._jreader.load(path))
  

In [0]:
results_df.write.jdbc(url=connectionUrl, table='dbo.results', properties=connectionProperties,mode='overwrite')

---------------------------------------------------------------------------
AnalysisException                         Traceback (most recent call last)
File <command-4477688624235572>, line 5
      1 orders_df=spark.read\
      2 .format('csv')\
      3 .option('header','true')\
      4 .option('inferSchema','true')\
----> 5 .load('dbfs:/mnt/sales/landing/{}'.format(filename))

File /databricks/spark/python/pyspark/instrumentation_utils.py:47, in _wrap_function.<locals>.wrapper(*args, **kwargs)
     45 start = time.perf_counter()
     46 try:
---> 47     res = func(*args, **kwargs)
     48     logger.log_success(
     49         module_name, class_name, function_name, time.perf_counter() - start, signature
     50     )
     51     return res

File /databricks/spark/python/pyspark/sql/readwriter.py:312, in DataFrameReader.load(self, path, format, schema, **options)
    310 self.options(**options)
    311 if isinstance(path, str):
--> 312     return self._df(self._jreader.load(path))
  